In [1]:
from datasets import load_dataset
data=load_dataset('lavita/ChatDoctor-HealthCareMagic-100k')

Found cached dataset parquet (/Users/vivianwang/.cache/huggingface/datasets/lavita___parquet/lavita--ChatDoctor-HealthCareMagic-100k-2ac509ec87e3b452/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
import pandas as pd
df=pd.DataFrame(data['train'])
     

df

,instruction,input,output
0,"If you are a doctor, please answer the medical...",I woke up this morning feeling the whole room ...,"Hi, Thank you for posting your query. The most..."
1,"If you are a doctor, please answer the medical...",My baby has been pooing 5-6 times a day for a ...,Hi... Thank you for consulting in Chat Doctor....
2,"If you are a doctor, please answer the medical...","Hello, My husband is taking Oxycodone due to a...","Hello, and I hope I can help you today.First, ..."
3,"If you are a doctor, please answer the medical...",lump under left nipple and stomach pain (male)...,HI. You have two different problems. The lump ...
4,"If you are a doctor, please answer the medical...",I have a 5 month old baby who is very congeste...,Thank you for using Chat Doctor. I would sugge...
...,...,...,...
112160,"If you are a doctor, please answer the medical...",im 25 years old ..i started using mtp kit on 5...,"Hello, Thanks for letting us know your health ..."
112161,"If you are a doctor, please answer the medical...",My 5 year old son has been coughing for a mont...,As you have mentioned in your history that you...
112162,"If you are a doctor, please answer the medical...",My toes on right foot more than left are numb ...,Hi. The numbness and blue discoloration could ...
112163,"If you are a doctor, please answer the medical...","I was diagnosis with pleurisy last Tuesday, an...",Thanks for your question on Chat Doctor. Treat...


In [3]:
df.shape

     

df = df.drop('instruction', axis=1)
     

df
     


,input,output
0,I woke up this morning feeling the whole room ...,"Hi, Thank you for posting your query. The most..."
1,My baby has been pooing 5-6 times a day for a ...,Hi... Thank you for consulting in Chat Doctor....
2,"Hello, My husband is taking Oxycodone due to a...","Hello, and I hope I can help you today.First, ..."
3,lump under left nipple and stomach pain (male)...,HI. You have two different problems. The lump ...
4,I have a 5 month old baby who is very congeste...,Thank you for using Chat Doctor. I would sugge...
...,...,...
112160,im 25 years old ..i started using mtp kit on 5...,"Hello, Thanks for letting us know your health ..."
112161,My 5 year old son has been coughing for a mont...,As you have mentioned in your history that you...
112162,My toes on right foot more than left are numb ...,Hi. The numbness and blue discoloration could ...
112163,"I was diagnosis with pleurisy last Tuesday, an...",Thanks for your question on Chat Doctor. Treat...


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.utils import to_categorical

# Load your data and preprocess it
# Assuming 'df' contains your preprocessed data

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['input'].values)

vocab_size = len(tokenizer.word_index) + 1
maxlen = 100  # Define the maximum sequence length

# Convert text to sequences
input_sequences = tokenizer.texts_to_sequences(df['input'].values)
output_sequences = tokenizer.texts_to_sequences(df['output'].values)

# Pad sequences to ensure uniform length
input_sequences = pad_sequences(input_sequences, maxlen=maxlen, padding='post')
output_sequences = pad_sequences(output_sequences, maxlen=maxlen, padding='post')

# One-hot encode the output sequences to align with model output layer
output_sequences_one_hot = to_categorical(output_sequences, num_classes=vocab_size)

# Create train-test split
input_train, input_test, output_train, output_test = train_test_split(
    input_sequences, output_sequences_one_hot, test_size=0.2, random_state=42
)

# Model creation
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



2023-12-04 23:24:22.640874: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Training the model
model.fit(input_train, output_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluating the model
loss, accuracy = model.evaluate(input_test, output_test)
print(f"Test Accuracy: {accuracy}")
